In [2]:
!pip install -q transformers

DEPRECATION: devscripts 2.22.1ubuntu1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of devscripts or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [34]:
import export_hf as exp
from pathlib import Path
from transformers import AutoModelForCausalLM, AutoConfig
import json

In [6]:
models = list(Path('/pfss/mlde/workspaces/mlde_wsp_MorphPiece/exps_llmc').glob('gpt3s_*/model*.bin'))

In [7]:
models

[PosixPath('/pfss/mlde/workspaces/mlde_wsp_MorphPiece/exps_llmc/gpt3s_bpe_50k/model_00005000.bin'),
 PosixPath('/pfss/mlde/workspaces/mlde_wsp_MorphPiece/exps_llmc/gpt3s_bpe_50k/model_00015000.bin'),
 PosixPath('/pfss/mlde/workspaces/mlde_wsp_MorphPiece/exps_llmc/gpt3s_bpe_50k/model_00010000.bin'),
 PosixPath('/pfss/mlde/workspaces/mlde_wsp_MorphPiece/exps_llmc/gpt3s_morph15k_50k/model_00005000.bin'),
 PosixPath('/pfss/mlde/workspaces/mlde_wsp_MorphPiece/exps_llmc/gpt3s_morph15k_50k/model_00015000.bin'),
 PosixPath('/pfss/mlde/workspaces/mlde_wsp_MorphPiece/exps_llmc/gpt3s_morph15k_50k/model_00010000.bin'),
 PosixPath('/pfss/mlde/workspaces/mlde_wsp_MorphPiece/exps_llmc/gpt3s_morph6k_50k/model_00005000.bin'),
 PosixPath('/pfss/mlde/workspaces/mlde_wsp_MorphPiece/exps_llmc/gpt3s_morph6k_50k/model_00015000.bin'),
 PosixPath('/pfss/mlde/workspaces/mlde_wsp_MorphPiece/exps_llmc/gpt3s_morph6k_50k/model_00010000.bin')]

In [39]:
def fix_attn_impl_config(model):
    config_file = f"{model}/config.json"
    c = json.load(open(config_file))
    c['_attn_implementation']="flash_attention_2"
    json.dump(c,open(f"{model}/config.json",'wt'), indent=2)

In [50]:
for bin_file in models[1:]:
    model_id = bin_file.parts[-2]
    step_k = int(bin_file.stem[9:])//1000
    output_id = f"{model_id}_step{step_k}k"
    exp.convert(filepath=bin_file,output=output_id)
    fix_attn_impl_config(output_id)

Converting model /pfss/mlde/workspaces/mlde_wsp_MorphPiece/exps_llmc/gpt3s_bpe_50k/model_00015000.bin to gpt3s_bpe_50k_step15k in bfloat16 format and pushing to Hugging Face: False
version=5, maxT=1024, V=50257, Vp=50304, L=12, H=12, C=768
Converting model /pfss/mlde/workspaces/mlde_wsp_MorphPiece/exps_llmc/gpt3s_bpe_50k/model_00010000.bin to gpt3s_bpe_50k_step10k in bfloat16 format and pushing to Hugging Face: False
version=5, maxT=1024, V=50257, Vp=50304, L=12, H=12, C=768
Converting model /pfss/mlde/workspaces/mlde_wsp_MorphPiece/exps_llmc/gpt3s_morph15k_50k/model_00005000.bin to gpt3s_morph15k_50k_step5k in bfloat16 format and pushing to Hugging Face: False
version=5, maxT=1024, V=50257, Vp=50304, L=12, H=12, C=768
Converting model /pfss/mlde/workspaces/mlde_wsp_MorphPiece/exps_llmc/gpt3s_morph15k_50k/model_00015000.bin to gpt3s_morph15k_50k_step15k in bfloat16 format and pushing to Hugging Face: False
version=5, maxT=1024, V=50257, Vp=50304, L=12, H=12, C=768
Converting model /pfs

In [21]:
exp.convert(filepath=models[0],output=output_id)

Converting model /pfss/mlde/workspaces/mlde_wsp_MorphPiece/exps_llmc/gpt3s_bpe_50k/model_00005000.bin to gpt3s_bpe_50k_step5k in bfloat16 format and pushing to Hugging Face: False
version=5, maxT=1024, V=50257, Vp=50304, L=12, H=12, C=768


In [46]:
model = AutoModelForCausalLM.from_pretrained('gpt3s_bpe_50k_step5k')

In [49]:
model.config

GPT2Config {
  "_name_or_path": "gpt3s_bpe_50k_step5k",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.41.2",
  "use_cache": true,
  "vocab_size": 50257
}

In [26]:
config._attn_implementation = "flash_attention_2"

In [40]:
fix_attn_impl_config('gpt3s_bpe_50k_step5k')

In [43]:
output_id

'gpt3s_bpe_50k_step5k'